In [38]:
import quandl
quandl.ApiConfig.api_key = "KxKMd8dsHx1sWnaGj6_U"
import numpy as np
import matplotlib as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler

In [51]:
def get_monthly_close_prices_ticker(ticker,start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SEP',ticker=ticker,date={'gte':start_date,'lte':end_date}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)
    close = df.pivot(index='date', columns='ticker', values='close')
    monthly = close.resample('BM').apply(lambda x: x[-1])
    #.agg(lambda x: x[-1])
    return monthly

In [52]:
monthly_prices = get_monthly_close_prices_ticker('A','2017-13-31','2019-01-01')

In [53]:
monthly_prices.head()

ticker,A
date,
2018-01-31,73.43
2018-02-28,68.59
2018-03-30,66.90
2018-04-30,65.74
2018-05-31,61.92


In [18]:
#quandl.get_table('SHARADAR/SEP',date={'gte':'2018-06-30','lte':'2019-01-01'}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)

In [54]:
def get_monthly_returns(monthly_prices):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
        

    #.agg(lambda x: x[-1])
    monthly_returns = (monthly_prices - monthly_prices.shift(1))/monthly_prices.shift(1).fillna(value=0)
    
    
    return monthly_returns

In [55]:
df1 = get_monthly_returns(monthly_prices)


In [56]:
df1.head()

ticker,A
date,
2018-01-31,NaN
2018-02-28,-0.065913
2018-03-30,-0.024639
2018-04-30,-0.017339
2018-05-31,-0.058108


In [57]:
df1 = df1.iloc[ 1: , : ]

In [58]:
df1.reset_index(inplace=True)


In [61]:
df1.head()

ticker,date,A
0,2018-02-28,-0.065913
1,2018-03-30,-0.024639
2,2018-04-30,-0.017339
3,2018-05-31,-0.058108
4,2018-06-29,-0.001292


In [64]:
df1['date'] = pd.to_datetime(df1['date']).dt.strftime("%Y-%m-%d")



In [65]:
df1['date'].resample('M')#.apply(lambda x: x[-1])


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [33]:
df1['calendardate'].resample.

close.resample('BM').apply(lambda x: x[-1])

ticker,date,A
0,2018-02-28,-0.065913
1,2018-03-30,-0.024639
2,2018-04-30,-0.017339
3,2018-05-31,-0.058108
4,2018-06-29,-0.001292
5,2018-07-31,0.067917
6,2018-08-31,0.022714
7,2018-09-28,0.044418
8,2018-10-31,-0.081514
9,2018-11-30,0.116685


In [24]:
def get_factor_data_ticker(ticker,start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SF1',ticker=ticker,dimension='ARQ',calendardate={'gte':start_date, 'lte':end_date}, paginate=True).fillna(value=0)

    
    return df

In [25]:
df2 = get_factor_data_ticker('A','2017-13-31','2019-01-01')

In [26]:
df2.head()

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,A,ARQ,2018-12-31,2019-03-05,2019-01-31,2019-03-05,-381000000,8952000000,0,3712000000,...,317515869,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000
1,A,ARQ,2018-09-30,2018-12-20,2018-10-31,2019-03-05,-408000000,8541000000,0,3848000000,...,318533054,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000
2,A,ARQ,2018-06-30,2018-08-30,2018-07-31,2019-03-05,-335000000,8349000000,0,3667000000,...,318769547,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000
3,A,ARQ,2018-03-31,2018-05-31,2018-04-30,2019-03-05,-310000000,8784000000,0,4525000000,...,319952126,322000000,326000000,3.745,5852000000,0,22000000,0,18.174,3160000000


In [27]:
df2['calendardate'] = pd.to_datetime(df2['calendardate']).dt.strftime("%Y-%m-%d")


In [34]:
def add_returns(row, df):
    c_date = row['calendardate']
    ticker = row['ticker']
    result = 0.0
    value = df.loc[df['date'] == c_date]
    if not value.empty:
        if ticker in value.columns:
            result = value[ticker].values[0]
    return result
df2['results'] = df2.apply(lambda row: add_returns(row, df1), axis=1)

In [37]:
df2.sort_values('calendardate')

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,results
None,,,,,,,,,,,,,,,,,,,,,
3,A,ARQ,2018-03-31,2018-05-31,2018-04-30,2019-03-05,-310000000,8784000000,0,4525000000,...,322000000,326000000,3.745,5852000000,0,22000000,0,18.174,3160000000,0.000000
2,A,ARQ,2018-06-30,2018-08-30,2018-07-31,2019-03-05,-335000000,8349000000,0,3667000000,...,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000,0.000000
1,A,ARQ,2018-09-30,2018-12-20,2018-10-31,2019-03-05,-408000000,8541000000,0,3848000000,...,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000,0.000000
0,A,ARQ,2018-12-31,2019-03-05,2019-01-31,2019-03-05,-381000000,8952000000,0,3712000000,...,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000,-0.067588


In [23]:
corr_matrix = df.corr().abs()

In [24]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [25]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [27]:
df2 = df2.drop(df2[to_drop], axis=1)

In [28]:
df2.shape

(8412, 87)

In [29]:
df2['calendardate'] = pd.to_datetime(df2['calendardate']).dt.strftime("%d/%m/%Y")

In [41]:
import os
os.getcwd()
os.chdir('C:\\Users\\Myles\\Downloads\\')

In [54]:
df2.to_csv('C:\\Users\\Myles\\Downloads\\factorapi.csv')

In [55]:
df1.to_csv('C:\\Users\\Myles\\Downloads\\returnsapi.csv')

In [32]:
def add_returns(row, df):
    c_date = row['calendardate']
    ticker = row['ticker']
    result = 0.0
    value = df.loc[df['date'] == c_date]
    if not value.empty:
        if ticker in value.columns:
            result = value[ticker].values[0]
    return result



In [33]:
df2.apply(lambda row: add_returns(row, df1), axis=1)

KeyError: ('date', 'occurred at index 0')

In [ ]:
df1.head()

In [ ]:
from sklearn.preprocessing import scale

# Remove unimportant features (weekdays)
train_features = train_features.iloc[:, :-4]
test_features = test_features.iloc[:, :-4]

# Standardize the train and test features
scaled_train_features = scale(train_features)
scaled_test_features = scale(test_features)

# Plot histograms of the 14-day SMA RSI before and after scaling
f, ax = plt.subplots(nrows=2, ncols=1)
train_features.iloc[:, 2].hist(ax=ax[0])
ax[1].hist(scaled_train_features[:, 2])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model_1 = Sequential()
model_1.add(Dense(100, input_dim=scaled_train_features.shape[1], activation='relu'))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dense(1, activation='linear'))

# Fit the model
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(scaled_train_features, train_targets, epochs=25)